# 02 Multi-Modality Fusion with GeoAnomalyMapper (GAM)

This tutorial builds on [01 Basic Analysis](01_basic_analysis.ipynb) to demonstrate fusing multiple geophysical data types (gravity, magnetic, seismic, InSAR) for more robust anomaly detection. We'll use joint inversion and advanced visualization.

**Objectives**:
- Combine all four modalities.
- Perform joint Bayesian fusion.
- Use advanced 3D visualization.
- Create custom workflows.

**Prerequisites**: Complete 01; ensure `[geophysics,visualization]` installed. Example uses Giza region.

In [ ]:
# Imports (same as 01)
import yaml
import pandas as pd
import numpy as np
from gam.core.pipeline import run_pipeline
from gam.visualization.manager import generate_visualization
from gam.modeling.fusion import FusionInverter  # For custom fusion
import gam
print(f"GAM version: {gam.__version__}")

## Step 1: Advanced Configuration for Fusion

Enable all modalities and tune fusion parameters (joint_weight for balancing).

In [ ]:
# Giza bbox (larger for multi-modal)
bbox = (29.9, 30.1, 31.1, 31.3)
modalities = ["gravity", "magnetic", "seismic", "insar"]  # All four

# Fusion-focused config
config = {
    "data": {
        "bbox": bbox,
        "modalities": modalities,
        "cache_dir": "../data/cache"
    },
    "preprocessing": {
        "grid_res": 0.005  # Finer for fusion accuracy
    },
    "modeling": {
        "inversion_type": "linear",  # Base for all
        "threshold": 2.5,
        "priors": {
            "joint_weight": 0.7,  # Higher weight for fusion
            "regularization": "l2"
        },
        "max_iterations": 100
    },
    "visualization": {
        "map_type": "3d",  # Advanced 3D
        "export_formats": ["vtk", "html", "csv"],
        "color_scheme": "plasma"
    },
    "core": {
        "output_dir": "../results/giza_fusion",
        "parallel_workers": 4
    }
}

with open("giza_fusion_config.yaml", "w") as f:
    yaml.dump(config, f)
print("Fusion config saved.")

## Step 2: Run Multi-Modal Pipeline

Fetch and process all modalities, then fuse.

In [ ]:
# Run full pipeline with fusion
results = run_pipeline(
    bbox=bbox,
    modalities=modalities,
    config=config,
    output_dir=config["core"]["output_dir"]
)

anomalies = results['anomalies']
fused_model = results['models']['fused']  # xarray.Dataset

print(f"Fusion complete. Anomalies: {len(anomalies)}")
print("Sample fused model stats:")
print(fused_model['data'].describe())

## Step 3: Joint Inversion Details

Inspect fusion contributions. Custom fusion example below.

In [ ]:
# View modality contributions in anomalies
print("Anomaly contributions example:")
print(anomalies[['lat', 'lon', 'confidence', 'modality_contributions']].head())

# Custom fusion (advanced: manual joint inversion)
from gam.modeling.fusion import FusionInverter
models = results['models']  # Dict of per-modality models
fusion = FusionInverter(joint_weight=config['modeling']['priors']['joint_weight'])
custom_fused = fusion.fuse(list(models.values()), priors=config['modeling']['priors'])
print("Custom fused model shape:", custom_fused['data'].shape)

# Detect on custom fused
from gam.modeling.anomaly_detection import AnomalyDetector
detector = AnomalyDetector(threshold=config['modeling']['threshold'])
custom_anomalies = detector.detect(custom_fused)
print(f"Custom detection: {len(custom_anomalies)} anomalies")

## Step 4: Advanced Visualization Techniques

Generate 3D volume and interactive map.

In [ ]:
# 3D visualization (VTK export for ParaView)
viz_paths = generate_visualization(
    custom_anomalies,
    type="3d",
    output_dir=config["core"]["output_dir"],
    config=config['visualization']
)

# Interactive 2D with contributions overlay
interactive_paths = generate_visualization(
    anomalies,
    type="interactive",
    color_by="modality_contributions.gravity",  # Example overlay
    output_dir=config["core"]["output_dir"]
)

print("Advanced viz saved:", list(viz_paths.values()))

# Quick plot of fused model slice (depth=0)
if 'depth' in fused_model.coords:
    slice_0 = fused_model.sel(depth=0, method='nearest')
    slice_0['data'].plot(cmap='plasma', figsize=(10, 8))
    plt.title("Fused Model Surface Slice")
    plt.show()

## Step 5: Custom Processing Workflows

Example: Custom workflow with post-fusion filtering.

In [ ]:
# Custom workflow: Fuse, filter by depth, re-visualize
def custom_workflow(bbox, modalities, config):
    # Run standard pipeline
    results = run_pipeline(bbox=bbox, modalities=modalities, config=config)
    
    # Custom: Filter anomalies >100m depth (subsurface)
    subsurface = results['anomalies'][results['anomalies']['depth'] < -100]
    
    # Re-run viz on filtered
    filtered_viz = generate_visualization(
        subsurface,
        type="3d",
        output_dir=f"{config['core']['output_dir']}/filtered"
    )
    
    return subsurface, filtered_viz

# Execute custom
subsurface_anoms, filtered_paths = custom_workflow(bbox, modalities, config)
print(f"Subsurface anomalies: {len(subsurface_anoms)}")
print("Filtered viz:", filtered_paths)

# Interpretation: Fusion reduces noise; gravity/seismic combo highlights voids better than single modality.

## Next Steps

- Explore global processing in [03 Global Processing](03_global_processing.ipynb).
- Tune priors for your use case (e.g., higher joint_weight for balanced fusion).
- Integrate with GIS: Load CSV/VTK in QGIS/ParaView.